In [10]:

# standard library imports
import git, os, sys

git_repo = git.Repo(os.getcwd(), search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")
os.chdir(f'{git_root}/src')
sys.path.append(os.path.abspath(os.path.join(f'{git_root}/src')))
print(f'Changed working directory to {os.getcwd()}')

# local imports
from fight_stat_helpers import *
from data_handler import DataHandler

Changed working directory to C:\Users\Alex\OneDrive\Documents\GitHub\UFC_Prediction_2022\src


# TODO import these functions from fight_stat_helpers like in the womens notebook

# NEW FEATURES

- Dominance Score for a single fight: (total score of fighter based on events in the fight) - (total score of opponent based on events in the fight)
- Average Dominance over time scales
- Fighter Scores
- Fight Math
- Fighter Score increments based on closeness of fights with opponents with high scores 

# Other things to try
- with or without regularization
- with or without scaling
- with or without sum features (seems like only age needs sum features)
- with or without squared features (seems like only age needs squared features)
- Decide whether to include SDEC in bullshit or not
- Do linear reg on draws and SDEC and see how this compares to log reg plane (hypothesis: they are the same plane since close fights should be near the plane boundary)

In [11]:
dh = DataHandler()
# get reported derived doubled data
ufc_fights_reported_derived_doubled = dh.get('ufc_fights_reported_derived_doubled')

In [12]:
ufc_fights_predictive_flattened_diffs = dh.make_ufc_fights_predictive_flattened_diffs(ufc_fights_reported_derived_doubled)
ufc_fights_predictive_flattened_diffs = dh.clean_ufc_fights_for_winner_prediction(ufc_fights_predictive_flattened_diffs)

In [13]:
# show division counts
print(ufc_fights_predictive_flattened_diffs['division'].value_counts())

division
Lightweight              1059
Welterweight             1016
Middleweight              820
Featherweight             669
Bantamweight              593
Light Heavyweight         578
Heavyweight               525
Flyweight                 305
Women's Strawweight       274
Women's Flyweight         222
Women's Bantamweight      178
Catch Weight               62
Women's Featherweight      26
Name: count, dtype: int64


In [14]:
# filter by division. start with just lightweight and featherweight
division_mask = ufc_fights_predictive_flattened_diffs['division'].isin(['Welterweight', 'Middleweight', 'Light Heavyweight', 'Heavyweight'])
ufc_fights_predictive_flattened_diffs_welter_heavy = ufc_fights_predictive_flattened_diffs[division_mask]
ufc_fights_predictive_flattened_diffs_welter_heavy.shape

(2939, 321)

In [15]:
# drop the 'fighter_result' column and opponent_result column as it is not needed for the model and also fighter and opponent columns
X = ufc_fights_predictive_flattened_diffs_welter_heavy.drop(columns=['fighter', 'opponent', 'method', 'division'])
# KEEP result in X as this is what model_score is expecting for better or for worse
y = X['result']

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=67)

# Use the entire data set with all features as a baseline

In [17]:
all_features = X_train.columns.tolist()
# remove result from all_features
if 'result' in all_features:
    all_features.remove('result')
model_test_score(X_train, X_test, all_features, _max_iter = 20000, scaled=True)

Training set size: (2351, 317) accuracy: 0.6712037430880476
Test set size: (588, 317) accuracy: 0.5969387755102041
Test set neg log loss: -0.6809233556918817. Probability to observe data given model: 0.5061494205828387


In [18]:
best_features = additive_greedy(X_train, X_test, search_doubles=True, _max_iter=30000)

Starting with an empty feature set.
Added single feature: l5y_overall_fighter_score_diff
Negative log loss on training set: -0.6818658092535146
Training set size: (2351, 317) accuracy: 0.5652913653764355
Test set size: (588, 317) accuracy: 0.5816326530612245
Test set neg log loss: -0.6834822586238917. Probability to observe data given model: 0.5048558890631086
Added single feature: l3y_wins_diff
Negative log loss on training set: -0.6754634148405187
Training set size: (2351, 317) accuracy: 0.5754997873245428
Test set size: (588, 317) accuracy: 0.5595238095238095
Test set neg log loss: -0.6885580304226199. Probability to observe data given model: 0.5022998482067301
Added single feature: l5y_wins_wins_diff
Negative log loss on training set: -0.6674881703983575
Training set size: (2351, 317) accuracy: 0.5852828583581454
Test set size: (588, 317) accuracy: 0.5697278911564626
Test set neg log loss: -0.6752376104511058. Probability to observe data given model: 0.5090354540927884
Added single

In [19]:
model_test_score(X_train, X_test, best_features, _max_iter = 30000, scaled=True)

Training set size: (2351, 317) accuracy: 0.654189706507869
Test set size: (588, 317) accuracy: 0.5816326530612245
Test set neg log loss: -0.6771460866439754. Probability to observe data given model: 0.5080648984830497
